In [2]:
from matdeeplearn.common import data
import os
import torch
import matplotlib.pyplot as plt
from pprint import pprint

from matdeeplearn.preprocessor.helpers import *
from matdeeplearn.preprocessor.transforms import *
from matdeeplearn.common.graph_data import CustomBatchingData

from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch
from ase import neighborlist

import cProfile, pstats, io
from pstats import SortKey

%pprint
%load_ext autoreload
%autoreload 2

torch.set_printoptions(profile="full")

Pretty printing has been turned ON
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import cProfile, pstats, io
from pstats import SortKey

In [4]:
pr1 = cProfile.Profile()
pr2 = cProfile.Profile()

In [5]:
dataset = data.get_dataset("/Users/sidbaskaran/desktop/research/hMOF/raw_5k")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/sidbaskaran/desktop/research/hMOF/raw_5k/data.pt'

In [14]:
dataset[0]

CustomBatchingData(n_atoms=[1], pos=[257, 3], cell=[3, 3], cell2=[6], y=64.5531005859375, z=[257], u=[1, 3], structure_id=[1])

In [7]:
dlist = [CustomBatchingData(d) for d in dataset[:50]]

In [8]:
batch_test = Batch.from_data_list(dlist)

In [9]:
batch_test.n_atoms

tensor([ 257,  136,  228,  235,  257,  142,  351,  384, 1204,  252,  108,  257,
         244,  344,  194,  286, 1175,  804,  623, 1077,  114,  619,  382,  102,
         300,  102,  796,  456,  346,  114,  150,  300,  372, 3147,  267,  822,
         176,  563,  478,  600,  126,  709,  266,  168,  340,  475,  352,  142,
         199,  141])

In [23]:
# non-batched performance test
pr1.enable()
for d in dlist:
    edge_index, unit_cell, num_neighbors_img = radius_graph_pbc(5.0, 250, d.pos, d.cell.view(-1, 3, 3), d.n_atoms)
pr1.disable()

In [25]:
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr1, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

         8596 function calls in 26.063 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000   26.237    6.559 /nethome/sbaskaran31/miniconda3/envs/matdeeplearn/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3397(run_code)
        4    0.000    0.000   26.236    6.559 {built-in method builtins.exec}
       60    2.432    0.041   24.894    0.415 /net/csefiles/coc-fung-cluster/Sidharth/MatDeepLearn_dev/matdeeplearn/preprocessor/helpers.py:997(radius_graph_pbc)
        1    1.165    1.165   23.358   23.358 /tmp/ipykernel_4079154/31310314.py:3(<module>)
      240   19.621    0.082   19.621    0.082 {built-in method torch.masked_select}
      180    0.999    0.006    0.999    0.006 {method 'repeat' of 'torch._C._TensorBase' objects}
      240    0.686    0.003    0.686    0.003 {built-in method torch.repeat_interleave}
      120    0.000    0.000    0.433    0.004 /nethome/sbaskaran31/minico

In [26]:
# batched performance test
pr2.enable()
edge_index, unit_cell, num_neighbors_img = radius_graph_pbc(5.0, 250, batch_test.pos, batch_test.cell, batch_test.n_atoms)
pr2.disable()

In [27]:
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr2, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())

         368 function calls in 22.440 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000   22.551    5.638 /nethome/sbaskaran31/miniconda3/envs/matdeeplearn/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3397(run_code)
        4    0.000    0.000   22.551    5.638 {built-in method builtins.exec}
        2    2.125    1.063   21.317   10.658 /net/csefiles/coc-fung-cluster/Sidharth/MatDeepLearn_dev/matdeeplearn/preprocessor/helpers.py:997(radius_graph_pbc)
        1    1.122    1.122   20.242   20.242 /tmp/ipykernel_4079154/2046315657.py:3(<module>)
        8   16.538    2.067   16.538    2.067 {built-in method torch.masked_select}
        6    1.018    0.170    1.018    0.170 {method 'repeat' of 'torch._C._TensorBase' objects}
        8    0.730    0.091    0.730    0.091 {built-in method torch.repeat_interleave}
        4    0.000    0.000    0.445    0.111 /nethome/sbaskaran31/minic

In [27]:
edge_index.shape

torch.Size([2, 38180])

In [72]:
torch.cumsum(torch.tensor([1,1,1,1]), dim=0)

tensor([1, 2, 3, 4])

In [17]:
a, b, c, alpha, beta, gamma = dataset[0].cell2
increment = 3

# obtain fractional spacings from 0 to 1 of the virtual atoms
ar1 = torch.arange(0, 1, increment / a)
ar2 = torch.arange(0, 1, increment / b)
ar3 = torch.arange(0, 1, increment / c)

# use meshgrid to obtain x,y,z, coordinates for the virtual atoms
xx, yy, zz = torch.meshgrid([ar1[:], ar2[:], ar3[:]], indexing="ij")
coords = torch.stack([xx.flatten(), yy.flatten(), zz.flatten()], dim=-1)

In [18]:
coords.shape

torch.Size([216, 3])

In [19]:
lattice_vectors = dataset[0].cell
cartesian_coords = torch.transpose(torch.matmul(lattice_vectors, coords.transpose(0, 1)), 0, 1)

In [20]:
cartesian_coords.shape

torch.Size([216, 3])

In [52]:
# no difference between the two
structure = Atoms(
    scaled_positions=coords, cell=dataset[0].cell, pbc=[1, 1, 1]
)
structure1 = Atoms(
    scaled_positions=coords, cell=dataset[0].cell2, pbc=[1, 1, 1]
)

In [53]:
ase_cartesian = torch.tensor(structure1.get_positions())

In [54]:
cartesian_ocp = frac_to_cart_coords(coords, torch.tensor([[a, b, c]]), torch.tensor([[alpha, beta, gamma]]), len(coords))

In [55]:
sorted_cartesian_ocp = cartesian_ocp[cartesian_ocp[:, 0].argsort()]
sorted_ase_cartesian = ase_cartesian[ase_cartesian[:, 0].argsort()]
sorted_cartesian_coords = cartesian_coords[cartesian_coords[:, 0].argsort()]

In [57]:
coords

tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.1804],
        [0.0000, 0.0000, 0.3609],
        [0.0000, 0.0000, 0.5413],
        [0.0000, 0.0000, 0.7218],
        [0.0000, 0.0000, 0.9022],
        [0.0000, 0.1804, 0.0000],
        [0.0000, 0.1804, 0.1804],
        [0.0000, 0.1804, 0.3609],
        [0.0000, 0.1804, 0.5413],
        [0.0000, 0.1804, 0.7218],
        [0.0000, 0.1804, 0.9022],
        [0.0000, 0.3609, 0.0000],
        [0.0000, 0.3609, 0.1804],
        [0.0000, 0.3609, 0.3609],
        [0.0000, 0.3609, 0.5413],
        [0.0000, 0.3609, 0.7218],
        [0.0000, 0.3609, 0.9022],
        [0.0000, 0.5413, 0.0000],
        [0.0000, 0.5413, 0.1804],
        [0.0000, 0.5413, 0.3609],
        [0.0000, 0.5413, 0.5413],
        [0.0000, 0.5413, 0.7218],
        [0.0000, 0.5413, 0.9022],
        [0.0000, 0.7218, 0.0000],
        [0.0000, 0.7218, 0.1804],
        [0.0000, 0.7218, 0.3609],
        [0.0000, 0.7218, 0.5413],
        [0.0000, 0.7218, 0.7218],
        [0.000

In [56]:
sorted_ase_cartesian

tensor([[-5.3847e-03,  7.9601e-03,  1.5000e+01],
        [-4.3078e-03,  6.3681e-03,  1.2000e+01],
        [-4.1523e-03,  3.0080e+00,  1.5000e+01],
        [-3.2308e-03,  4.7761e-03,  9.0000e+00],
        [-3.0754e-03,  3.0064e+00,  1.2000e+01],
        [-2.9200e-03,  6.0080e+00,  1.5000e+01],
        [-2.1539e-03,  3.1841e-03,  6.0000e+00],
        [-1.9985e-03,  3.0048e+00,  9.0000e+00],
        [-1.8431e-03,  6.0064e+00,  1.2000e+01],
        [-1.6877e-03,  9.0080e+00,  1.5000e+01],
        [-1.0769e-03,  1.5920e-03,  3.0000e+00],
        [-9.2153e-04,  3.0032e+00,  6.0000e+00],
        [-7.6613e-04,  6.0048e+00,  9.0000e+00],
        [-6.1073e-04,  9.0064e+00,  1.2000e+01],
        [-4.5532e-04,  1.2008e+01,  1.5000e+01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.5540e-04,  3.0016e+00,  3.0000e+00],
        [ 3.1081e-04,  6.0032e+00,  6.0000e+00],
        [ 4.6621e-04,  9.0048e+00,  9.0000e+00],
        [ 6.2162e-04,  1.2006e+01,  1.2000e+01],
        [ 7.7702e-04

In [45]:
sorted_cartesian_ocp

tensor([[ 0.0000e+00,  0.0000e+00,  3.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  6.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  9.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  1.2000e+01],
        [ 0.0000e+00,  0.0000e+00,  1.5000e+01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.2329e-03,  3.0000e+00,  1.5917e-03],
        [ 1.2329e-03,  3.0000e+00,  3.0016e+00],
        [ 1.2329e-03,  3.0000e+00,  6.0016e+00],
        [ 1.2329e-03,  3.0000e+00,  9.0016e+00],
        [ 1.2329e-03,  3.0000e+00,  1.2002e+01],
        [ 1.2329e-03,  3.0000e+00,  1.5002e+01],
        [ 2.4657e-03,  6.0000e+00,  3.0032e+00],
        [ 2.4657e-03,  6.0000e+00,  1.5003e+01],
        [ 2.4657e-03,  6.0000e+00,  1.2003e+01],
        [ 2.4657e-03,  6.0000e+00,  9.0032e+00],
        [ 2.4657e-03,  6.0000e+00,  6.0032e+00],
        [ 2.4657e-03,  6.0000e+00,  3.1833e-03],
        [ 3.6986e-03,  9.0000e+00,  4.7750e-03],
        [ 3.6986e-03,  9.0000e+00,  3.0048e+00],
        [ 3.6986e-03

In [3]:
batches = DataLoader(dataset[:4], batch_size=4, shuffle=False)

In [4]:
len(batches)

1

In [5]:
batch = next(iter(batches))

In [12]:
batch.cell2

[[16.625548, 16.625546, 16.625539, 89.969603, 90.02056800000001, 89.976464], [10.489711, 10.489711, 9.016243999999999, 90.001499, 89.99110600000002, 84.52700900000002], [17.699077, 16.776229, 9.065218000000002, 89.99523399999998, 95.851769, 90.006585], [16.625548, 12.64533, 16.625539000000003, 89.97574899999998, 89.979432, 90.01876]]